# Analysis

The policy paper outlines four different UBI levels, each with different net costs after UBI and tax-benefit changes.

## Reform Specification

Below specifies the first reform, and calculates the effects of each component. This is a £45/week UBI for working-age adults, funded in part by including it in benefit means tests, reducing the Personal Allowance for working-age adults and reducing the National Insurance Primary Threshold to 

In [1]:
from openfisca_uk import Microsimulation
from reform import WA_adult_UBI, include_UBI_in_means_tests, set_PA, set_PA_for_WA_adults, set_PT, net_cost
import numpy as np
import pandas as pd
from tqdm import trange

ubi_45 = (WA_adult_UBI(45 * 52), include_UBI_in_means_tests(), set_PA_for_WA_adults(2500), set_PT(50))
baseline = Microsimulation()
ubi_45_sim = Microsimulation(ubi_45)

net_costs = []
names = ["UBI (£45/week)", "Reduced Benefits", "Lower PA", "Lower PT"]
for i in range(len(ubi_45) + 1):
    net_costs += [net_cost(baseline, Microsimulation(ubi_45[:i]))]
net_costs = np.array(net_costs)

net_cost = pd.Series((net_costs[1:] - net_costs[:-1]) / 1e+9).apply(lambda x: round(x, 1))

pd.DataFrame({"Component": names, "Net cost (£bn)": net_cost, "Resulting net cost": net_cost.cumsum()}).set_index("Component")

,Net cost (£bn),Resulting net cost
Component,,
UBI (£45/week),93.6,93.6
Reduced Benefits,-6.0,87.6
Lower PA,-58.6,29.0
Lower PT,-37.2,-8.2
